# Chat Over Tabular Data - Pipeline Demo

This notebook walks through the full RAG-style pipeline step by step, with prints at each stage so you can see how each part works and experiment.

## 1. Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

from src.data_loader import load_all_tables, get_sqlite_connection
from src.schema import build_schema_prompt
from src.text_to_sql import generate_sql
from src.sql_validator import validate_sql
from src.executor import execute_query
from src.llm_client import generate_answer
from src.pipeline import run_pipeline

print("✓ Imports loaded")

✓ Imports loaded


## 2. Step 1: Load Data

In [2]:
tables = load_all_tables()

print("Loaded tables:")
for name, df in tables.items():
    print(f"  - {name}: {len(df)} rows, columns: {list(df.columns)}")

conn = get_sqlite_connection(tables)
print("\n✓ In-memory SQLite DB created")

Loaded tables:
  - Clients: 20 rows, columns: ['client_id', 'client_name', 'industry', 'country']
  - Invoices: 40 rows, columns: ['invoice_id', 'client_id', 'invoice_date', 'due_date', 'status', 'currency', 'fx_rate_to_usd']
  - InvoiceLineItems: 96 rows, columns: ['line_id', 'invoice_id', 'service_name', 'quantity', 'unit_price', 'tax_rate']

✓ In-memory SQLite DB created


## 3. Step 2: Build Schema (Four Knowledge Types for LLM)

The schema prompt injects **four types of knowledge** before SQL generation:
1. **Structural** – tables, columns, data types
2. **Relational** – FKs, join paths, cardinality
3. **Semantic** – table/column meaning, business rules
4. **Usage constraints** – allowed joins, aggregation, time semantics

In [3]:
schema_prompt = build_schema_prompt(tables)

print("Schema sent to Text-to-SQL LLM:")
print("=" * 60)
print(schema_prompt)
print("=" * 60)

Schema sent to Text-to-SQL LLM:
You are a SQL expert. Write SQLite SQL queries using the knowledge below.
Use ALL four knowledge types: structural, relational, semantic, and usage constraints.

1. STRUCTURAL KNOWLEDGE (Schema)

**Clients** (20 rows)  # Business clients/customers. Each row is one client.

  - client_id (object): samples: 'C001', 'C002', 'C003', 'C004', 'C005'  # Client identifier (PK in Clients, FK in Invoices)
  - client_name (object): samples: 'Acme Corp', 'Bright Legal', 'Summit Finance', 'GreenField Retail', 'Northwind Logistics'
  - industry (object): samples: 'Manufacturing', 'Legal', 'Financial Services', 'Retail', 'Logistics'  # Industry sector (e.g. Legal, Tech)
  - country (object): samples: 'USA', 'UK', 'Germany', 'Canada', 'Australia'  # Client country. Use COUNTRY_REGIONS for 'European', 'Americas', etc.

**Invoices** (40 rows)  # Invoices issued to clients. Each row is one invoice. Links to Clients.

  - invoice_id (object): samples: 'I1001', 'I1002', 'I10

## 4. Step 3: Generate SQL (Text-to-SQL)

In [4]:
import os 
from openai import OpenAI

def generate_sql(question: str, tables: dict, model: str = "gpt-4o-mini") -> str:
    """Use OpenAI to generate a SQL query from a natural language question."""
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    schema_text = build_schema_prompt(tables)

    system = f"""{schema_text}

When the user asks a question, respond with ONLY a valid SQLite SQL query.
No markdown, no explanation, no backticks. Just the raw SQL."""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": question},
        ],
        temperature=0,
    )
    sql = response.choices[0].message.content.strip()
    # Remove markdown code blocks if present
    if sql.startswith("```"):
        lines = sql.split("\n")
        sql = "\n".join(
            line for line in lines if not line.strip().startswith("```")
        )
    return sql


In [6]:
question = 'Which client has the highest total billed amount in 2024, and what is that total?'


print(f"Question: {question}")
print("\nCalling OpenAI to generate SQL...")

sql = generate_sql(question, tables)

print(f"\nGenerated SQL:\n{sql}")

Question: Which client has the highest total billed amount in 2024, and what is that total?

Calling OpenAI to generate SQL...

Generated SQL:
SELECT Clients.client_name, SUM(InvoiceLineItems.quantity * InvoiceLineItems.unit_price) AS total_billed
FROM Clients
JOIN Invoices ON Clients.client_id = Invoices.client_id
JOIN InvoiceLineItems ON Invoices.invoice_id = InvoiceLineItems.invoice_id
WHERE strftime('%Y', Invoices.invoice_date) = '2024'
GROUP BY Clients.client_name
ORDER BY total_billed DESC
LIMIT 1;


## 5. Step 4: Validate SQL (before execution)

In [6]:
valid, error_msg = validate_sql(sql, conn)

if valid:
    print("✓ Validation passed (syntax, safety, schema)")
else:
    print(f"✗ Validation failed: {error_msg}")

✓ Validation passed (syntax, safety, schema)


## 6. Step 5: Execute Query

In [7]:
result = execute_query(conn, sql)

if isinstance(result, str):
    print(f"Error: {result}")
else:
    print(f"Rows returned: {len(result)}")
    print("\nResult:")
    display(result)

Rows returned: 2

Result:


,client_name
0,Bright Legal
1,Silverline Media


## 7. Step 6: Generate Answer (LLM)

In [8]:
if isinstance(result, str):
    print(f"Cannot generate answer - execution failed: {result}")
else:
    data_str = result.to_string() if len(result) > 0 else "(No rows returned)"

    print("Data passed to answer LLM (first 500 chars):")
    # print(data_str[:500] + "..." if len(data_str) > 500 else data_str)
    print("\nGenerating answer...")

    answer = generate_answer(question, data_str)

    print(f" {answer}")

Data passed to answer LLM (first 500 chars):

Generating answer...
retrieved data
        client_name
0      Bright Legal
1  Silverline Media
 The clients based in the UK are Bright Legal and Silverline Media.


## 8. Full Pipeline (run everything)

In [ ]:
# Change this to experiment with different questions
questions_list  = ['For each service_name in InvoiceLineItems, how many line items are there?',
'List all invoices for Acme Corp with their invoice IDs, invoice dates, due dates, and statuses.',
'Show all invoices issued to Bright Legal in February 2024, including their status and currency.',
'For invoice I1001, list all line items with service name, quantity, unit price, tax rate, and compute the line total (including tax) for each.',
'For each client, compute the total amount billed in 2024 (including tax) across all their invoices.',
'Which client has the highest total billed amount in 2024, and what is that total?',
'Across all clients, which three services generated the most revenue in 2024? Show the total revenue per service.',
'Which invoices are overdue as of 2024-12-31? List invoice ID, client name, invoice_date, due_date, and status.',
'Group revenue by client country: for each country, compute the total billed amount in 2024 (including tax).',
'For the service “Contract Review”, list all clients who purchased it and the total amount they paid for that service (including tax).',
'Considering only European clients, what are the top 3 services by total revenue (including tax) in H2 2024 (2024-07-01 to 2024-12-31)?']
for question in questions_list:

    print(f"Question: {question}")
    print("\nRunning full pipeline...")

    result = run_pipeline(question)

    print("\n--- Result ---")
    print(f"SQL: {result['sql']}")
    print(f"Validation passed: {result.get('validation_passed', 'N/A')}")
    if result.get('error'):
        print(f"Error: {result['error']}")
    else:
        print(f"Rows: {len(result['data'])}")
        print(f"Answer: {result['answer']}")
    

Question: Which client has the highest total billed amount in 2024, and what is that total?

Running full pipeline...
retrieved data
  client_name  total_billed
0   Acme Corp          6400

--- Result ---
SQL: SELECT Clients.client_name, SUM(InvoiceLineItems.quantity * InvoiceLineItems.unit_price) AS total_billed
FROM Clients
JOIN Invoices ON Clients.client_id = Invoices.client_id
JOIN InvoiceLineItems ON Invoices.invoice_id = InvoiceLineItems.invoice_id
WHERE strftime('%Y', Invoices.invoice_date) = '2024'
GROUP BY Clients.client_id
ORDER BY total_billed DESC
LIMIT 1;
Validation passed: True
Rows: 1
Answer: The client with the highest total billed amount in 2024 is Acme Corp, with a total of $6,400.
